In [1]:
using System;
using System.Collections.Generic;


public class User
{
    public int UserId { get; set; }
    public string Username { get; set; }
    public string Password { get; set; }
    public string Email { get; set; }
    public string Address { get; set; }
    public string PhoneNumber { get; set; }

    public void Register()
    {
        Console.WriteLine($"{Username} зарегистрирован.");
    }

    public bool Login(string password)
    {
        return password == Password;
    }

    public void UpdateProfile(string email, string address, string phoneNumber)
    {
        Email = email;
        Address = address;
        PhoneNumber = phoneNumber;
        Console.WriteLine("Профиль обновлен.");
    }
}

public class OrderLine
{
    public string Category { get; set; }
    public string Description { get; set; }  
    protected int ProductId { get; set; }
    protected string ProductName { get; set; }
    protected decimal Price { get; set; }
    protected int StockQuantity { get; set; }
    protected bool IsAvalible { get; set; }

    public virtual bool IsInStock()
    {
        return StockQuantity>0;
    }

    public int Productid
    {
        get { return ProductId; }
        set { ProductId = value; }
    }

    public string Productname
    {
        get { return ProductName; }
        set { ProductName = value; }
    }

    public decimal price
    {
        get { return Price; }
        set { Price = value; }
    }

    public OrderLine(int productId, string productName, decimal price, string category, string description)
    {
        ProductId = productId;
        ProductName = productName;
        Price = price;
        Category = category;
        Description = description;
    }

    public virtual decimal CalculateTotal()
    {
        return Price;
    }

    public void UpdatePrice(decimal newPrice)
    {
        Price = newPrice;
    }

    public virtual string GetProductDetails()
    {
        return $"Id товара: {ProductId}, Наименование: {ProductName}, Стоимость: {Price}";
    }

    public void CheckStock()  
    {
        Console.WriteLine($"{ProductName} в наличии"); 
    }

    public static OrderLine operator + (OrderLine line1, OrderLine line2)
    {
        return new OrderLine
        (
            line1.ProductId + line2.ProductId,
            line1.ProductName + line2.ProductName,
            line1.Price + line2.Price,
            line1.Category + line2.Category,
            line1.Description + line2.Description
        );
    }
}

class OrderLineWithDiscount : OrderLine
{
    protected decimal Discount { get; set; }

    public OrderLineWithDiscount(int productId, string productName, decimal price, string category, string description, decimal discount)
        : base(productId, productName, price, category, description)
    {
        Discount = discount;
    }

    public void ApplyDiscount()
    {
        decimal discountAmount = Price * Discount / 100;
        Price -= discountAmount;
    }

    public override string GetProductDetails()
    {
        return base.GetProductDetails() + $", Скидка: {Discount}";
    }
}

class StandertLine : OrderLineWithDiscount
{
    protected int Units { get; set; }

    public StandertLine(int productId, string productName, decimal price, string category, string description, decimal discount, int units)
        : base(productId, productName, price, category, description, discount)
    {
        Units = units;
    }

    public int units
    {
        get { return Units; }
        set { Units = value; }
    }

    public override decimal CalculateTotal()
    {
        return Price * Units;
    }

    public override string GetProductDetails()
    {
        return base.GetProductDetails() + $", Количество: {Units}";
    }
}

interface IReservable
{
    void Reserve();
}

class SpecialLine : OrderLineWithDiscount, IReservable
{
    public SpecialLine(int productId, string productName, decimal price, string category, string description, decimal extraDiscount)
        : base(productId, productName, price, category, description, extraDiscount)
    {
    }

    public override string GetProductDetails()
    {
        return base.GetProductDetails() + $", Максимальная скидка: {Discount}";
    }

    public void Reserve()
    {
        Console.WriteLine($"Забронировано {ProductName}");
    }
}

class FreeLine : OrderLineWithDiscount, IReservable
{
    protected decimal Prepayment { get; set; }  

    public FreeLine(int productId, string productName, decimal price, string category, string description, decimal prepayment, decimal discount)
        : base(productId, productName, price, category, description, discount)
    {
        Prepayment = prepayment;
    }

    public override decimal CalculateTotal()
    {
        return Price - Prepayment;
    }

    public override string GetProductDetails()
    {
        return base.GetProductDetails() + $", Предоплата: {Prepayment}";
    }

    public void Reserve()
    {
        Console.WriteLine($"Забронировано {ProductName}");
    }
}

class Order<T> where T :  OrderLine
{
    private List<T> orderLines = new List<T>();
    public void AddOrderLine(T orderline)
    {
        orderLines.Add(orderline);
    }
    public void PringOrderDetails()
    {
        foreach(var orderline in orderLines)
        {
            Console.WriteLine(orderline.GetProductDetails());
        }
    }
    
    public decimal CalculateTotalOrder()
    {
        decimal order=0;
        foreach(var orderline in  orderLines)
        {
            order += orderline.CalculateTotal();
        }
        return order;
    }
}

var user = new User { UserId = 1, Username = "user1", Password = "12345", Email = "user1@mail.ru" };
user.Register();

StandertLine standertLine = new StandertLine(1, "Кофта", 10, "Одежда", "Женское", 2, 5);
SpecialLine specialLine = new SpecialLine(2, "Джинсы", 15, "Одежда", "Мужское", 3);
FreeLine freeLine = new FreeLine(3, "Платье", 20, "Одежда", "Женское", 10, 10);

Order<OrderLine> order1 = new Order<OrderLine> ();
order1.AddOrderLine(standertLine);
order1.AddOrderLine(specialLine);
order1.AddOrderLine(freeLine);

order1.PringOrderDetails();
Console.WriteLine(order1.CalculateTotalOrder());

freeLine.Reserve();

OrderLine orderLine1 = new OrderLine(1, "Кофта", 100, "Одежда", "Женская");
OrderLine orderLine2 = new OrderLine(2, "Джинсы", 150, "Одежда", "Мужское");
OrderLine comb = orderLine1 + orderLine2;

Console.WriteLine(comb.GetProductDetails());

user1 зарегистрирован.
Id товара: 1, Наименование: Кофта, Стоимость: 10, Скидка: 2, Количество: 5
Id товара: 2, Наименование: Джинсы, Стоимость: 15, Скидка: 3, Максимальная скидка: 3
Id товара: 3, Наименование: Платье, Стоимость: 20, Скидка: 10, Предоплата: 10
75
Забронировано Платье
Id товара: 3, Наименование: КофтаДжинсы, Стоимость: 250
